In [12]:
from ckks_package.ckks import *

In [13]:
N = 2**15  # Ring degree
n = 2**2  # Number of slots
L = 20  # Maximal level
q0 = 2**40  # Smallest modulus
delta = 2**33  # Scaling factor

# The grouping parameter for bootstrapping is s. It lies between 1 and
# log(n, 2). The smaller s, the faster the bootstrapping, but the more levels
# are consumed.
s = log(n, 2)

CKKS.config(N, n, L, q0, delta)
CKKS.key_gen()
CKKS.config_bootstrap(CKKS.sk, s)

The CKKS configuration is done!

The key generation is done!
Estimated security: 2^(80.0).

Encoding relevant vectors as polynomials...
Generating matrices required for CoeffToSlot and SlotToCoeff...
Encoding these matrices as polynomials...
Generating missing switching keys...
The bootstrapping configuration is done!



In [3]:
# Creating two random complex vectors

complex_vectors = [
    np.array([randint(-3, 3) + 1j * randint(-3, 3) for _ in range(n)])
    for _ in range(2)
]
complex_vectors

[array([-2.+3.j,  2.-1.j, -0.-2.j,  2.-1.j]),
 array([ 0.+2.j, -1.+3.j, -3.+3.j, -2.-3.j])]

In [4]:
# Encoding as polynomials

plaintext_polys = [CKKS.encode(z) for z in complex_vectors]
plaintext_polys

[4294967296 + 140999689X^4096 - 4555500750X^8192 + 8276468117X^12288 - 2147483648X^16384 + 11563693771X^20480 + 13666502250X^24576 + 8077064444X^28672 mod(2^700),
 - 12884901888 + 16212532872X^4096 + 7592501250X^8192 - 2465419240X^12288 + 10737418240X^16384 - 11364290098X^20480 + 7592501250X^24576 + 5952048566X^28672 mod(2^700)]

In [5]:
# Encrypting

ciphertexts = [CKKS.enc_poly_with_sk(pt, CKKS.sk) for pt in plaintext_polys]
ciphertexts

[A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20),
 A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)]

In [6]:
# Checking correctness

for i in range(2):
    pt = ciphertexts[i].dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[-2.00000000e+00+3.j  2.00000000e+00-1.j -5.39989498e-10-2.j
  2.00000000e+00-1.j]
[-2.79763546e-10+2.j -1.00000000e+00+3.j -3.00000000e+00+3.j
 -2.00000000e+00-3.j]


In [7]:
# Homomorphic operations

ct_add = ciphertexts[0] + ciphertexts[1]
ct_mul = (
    ciphertexts[0] @ ciphertexts[1]
)  # Polynomial multiplication followed by a rescaling operation
print(ct_add)
print(ct_mul)

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^19 (level 19 out of 20)


In [8]:
# Checking correctness

for ct in [ct_add, ct_mul]:
    pt = ct.dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[-2.00000000e+00+5.j  9.99999999e-01+2.j -3.00000000e+00+1.j
  3.27070926e-10-4.j]
[-6.        -4.00000001j  1.        +7.j          6.        +6.j
 -7.00000001-4.j        ]


In [9]:
# Bootstrapping

ct = ciphertexts[0] % q0  # Project to the lowest level l = 0
print(ct)

ct_boot = ct.bootstrap(s)
print(ct_boot)

pt = ct_boot.dec_to_poly(CKKS.sk)
print(CKKS.decode(pt))

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^0 (level 0 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^2 (level 2 out of 20)
[-1.99828590e+00+2.99766625j  1.99923442e+00-0.99866092j
  4.11429434e-04-1.99970507j  1.99843446e+00-0.99927837j]
